In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

# constant seed
tf.random.set_seed(6950)


# Load the Diagnostics.xlsx data
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
diagnostics_df = pd.read_excel(diagnostics_file)

# Rename "SA" to "SI" in the "Rhythm" column
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].replace("SA", "SI")

# Drop rows with any missing values
diagnostics_df = diagnostics_df.dropna()

# Encode "Gender" column: 0 for "MALE" and 1 for "FEMALE"
diagnostics_df["Gender"] = diagnostics_df["Gender"].map({"MALE": 0, "FEMALE": 1})

# Merge specified labels
merge_mapping = {
    "AF": "AFIB",
    "AFIB": "AFIB",
    "SVT": "GSVT",
    "AT": "GSVT",
    "SAAWR": "GSVT",
    "ST": "GSVT",
    "AVNRT": "GSVT",
    "AVRT": "GSVT",
    "SB": "SB",
    "SR": "SR",
    "SI": "SR"
}
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].map(merge_mapping)

# Separate features and labels
features = diagnostics_df.drop(columns=["FileName", "Rhythm", "Beat"]).values
labels = diagnostics_df["Rhythm"].values  # Using "Rhythm" as the target variable

# Convert features to float32
features = features.astype("float32")

# Encode labels as one-hot with merged classes
unique_labels = np.unique(labels)
label_map = {label: index for index, label in enumerate(unique_labels)}
labels_encoded = to_categorical([label_map[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

2025-01-09 16:39:21.020308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 16:39:21.031287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 16:39:21.034683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 16:39:21.044283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-09 16:39:21.598656: W tensorflow/compiler/tf2

In [2]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8516, 13) (8516, 4)
(2130, 13) (2130, 4)


In [3]:
mlp = Sequential([
    Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(16, activation="relu"),
    Dense(labels_encoded.shape[1], activation="softmax")
])

mlp.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

mlp.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2,
        # callbacks=[early_stopping]
        )

# Evaluate the model
y_pred = mlp.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Map back to original labels for a readable report
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]
print("\nClassification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names, digits=5))


Epoch 1/500


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1736419163.327093  227904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736419163.359221  227904 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736419163.

 96/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3996 - loss: 1.3267

I0000 00:00:1736419164.775840  228056 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.4133 - loss: 1.3031 - val_accuracy: 0.5376 - val_loss: 1.3891
Epoch 2/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.7193 - loss: 0.7234 - val_accuracy: 0.7218 - val_loss: 0.6795
Epoch 3/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.8122 - loss: 0.5251 - val_accuracy: 0.7975 - val_loss: 0.5085
Epoch 4/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.8272 - loss: 0.4611 - val_accuracy: 0.7811 - val_loss: 0.5421
Epoch 5/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.8330 - loss: 0.4366 - val_accuracy: 0.8269 - val_loss: 0.4354
Epoch 6/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8382 - loss: 0.4245 - val_accuracy: 0.8187 - val_loss: 0.4253
Epoch 7/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.8396 - loss: 0.4185 - val_accuracy: 0.8386 - val_loss: 0.3881
Epoch 8/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.8435 - loss: 0.4121 - val_acc

In [4]:
import os

converter = tf.lite.TFLiteConverter.from_keras_model(mlp)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()

os.makedirs('GeneratedCHeaderFile', exist_ok=True)

# Save the TFLite model
tflite_model_path = 'GeneratedCHeaderFile/optimized_mlp.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

header_file_path = 'GeneratedCHeaderFile/optimized_mlp.h'
xxd_command = f'xxd -i {tflite_model_path} {header_file_path}'

import subprocess
subprocess.run(xxd_command, shell=True)


with open(header_file_path, 'r') as f:
    header_content = f.read()


modified_content = header_content.replace(
    'unsigned char GeneratedCHeaderFile_model_tflite[] = {',
    'const unsigned char model_data[] __attribute__((aligned(8))) = {'
)
modified_content = modified_content.replace(
    'unsigned int GeneratedCHeaderFile_model_tflite_len',
    'const unsigned int model_data_len'
)

# Write the modified content back
with open(header_file_path, 'w') as f:
    f.write(modified_content)

print(f"TFLite model saved to: {tflite_model_path}")
print(f"C header file saved to: {header_file_path}")

# Verify the model size
print(f"\nModel size: {os.path.getsize(tflite_model_path) / 1024:.2f} KB")

INFO:tensorflow:Assets written to: /tmp/tmp3dmqzjhy/assets


INFO:tensorflow:Assets written to: /tmp/tmp3dmqzjhy/assets


Saved artifact at '/tmp/tmp3dmqzjhy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  134089063521744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063520336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063521568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063524560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063520512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063518752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063525440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063528256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063525616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134089063529840: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved to: Ge

W0000 00:00:1736419688.775403  227904 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1736419688.775422  227904 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-01-09 16:48:08.775631: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp3dmqzjhy
2025-01-09 16:48:08.776024: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-09 16:48:08.776032: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp3dmqzjhy
2025-01-09 16:48:08.779688: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-01-09 16:48:08.780241: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-09 16:48:08.798689: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp3dmqzjhy
2025-01-09 16:48:08.804136: I tensorflow/cc/saved_model/loader.cc